In [2]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [3]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install vaderSentiment
    !pip install sentencepiece
    !python -m spacy download en_core_web_lg

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 3.3 MB/s 
     |████████████████████████████████| 182 kB 36.6 MB/s 
     |████████████████████████████████| 7.6 MB 54.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 56.1 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-py

In [4]:
import warnings
warnings.filterwarnings('ignore')

import re, os
import pandas as pd
import numpy as np
from collections import defaultdict
import xml.etree.ElementTree as ET

from sklearn.metrics.pairwise import rbf_kernel

import torch
from transformers import AutoTokenizer, BertTokenizerFast, BertModel, BertConfig

# import nltk
# from nltk import Tree

# import stanza
# from textblob import TextBlob
# from allennlp.predictors.predictor import Predictor

import spacy
from spacy import displacy
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex
nlp = spacy.load('en_core_web_lg')

In [5]:
# Set spacy
infixes = (
        LIST_ELLIPSES
        + LIST_ICONS
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
                al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
            ),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        ]
    )
    
infix_re = compile_infix_regex(infixes)
nlp = spacy.load('en_core_web_lg')
nlp.tokenizer.infix_finditer = infix_re.finditer

In [15]:
# Set directory
if IN_COLAB:
    root_path = '/content/drive/Othercomputers/DT Laptop/Sentiment Analysis/UnsupervisedABSA'
else:
    root_path = os.getcwd()

# Set root path
os.chdir(root_path)

In [12]:
def parse_xml_data(root):
    """Parses xml data to extract aspect terms and aspect categories from each sentence"""
    
    reviews = []

    for sentence in root.findall("sentence"):
        entry = {}
        aspect_terms = []
        aspect_categories = []
        
        if sentence.find("aspectTerms"):
            for aspect_term in sentence.find("aspectTerms").findall("aspectTerm"):
                aspect_terms.append((aspect_term.get("term"), aspect_term.get("polarity")))
                
        if sentence.find("aspectCategories"):
            for aspect_category in sentence.find("aspectCategories").findall("aspectCategory"):
                aspect_categories.append((aspect_category.get("category"), aspect_category.get("polarity")))
                
        entry["text"] = sentence[0].text
        entry["terms"] = aspect_terms
        entry["aspects"] = aspect_categories
        
        reviews.append(entry)

    reviews_df = pd.DataFrame(reviews)
    
    return reviews_df

In [17]:
# # Initialize pretrained models for subtasks
# coref_res_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
# coreference_resolver = Predictor.from_path(coref_res_url)

# stanzanlp = stanza.Pipeline(lang='fr', processors='tokenize,mwt,pos,lemma,depparse')

tokenizer = BertTokenizerFast.from_pretrained("./AutoABSA/rest_pft")
model = BertModel.from_pretrained("./AutoABSA/rest_pft", output_hidden_states=True)

Some weights of the model checkpoint at ./AutoABSA/rest_pft were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./AutoABSA/rest_pft and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You

In [18]:
# Parsing XML tree
tree = ET.parse('./semeval14/ABSA_TrainData/Restaurants_Train.xml')
root = tree.getroot()
df = parse_xml_data(root)

In [19]:
df.shape

(3044, 3)

In [20]:
# def resolve_coreference(text, remove_words = ['the']):
#     coref_res_pred = coreference_resolver.predict(text)
#     coref_resolved_text = coreference_resolver.coref_resolved(text)
#     for cluster in coref_res_pred['clusters']:
#         coref_word, coref_dep = cluster[0], cluster[1]
#         coref_list = text.split()[coref_word[0]:coref_word[1]+1]
#         for bad_word in remove_words:
#             try:
#                 coref_list.remove(bad_word)
#             except:
#                 pass
#         coref_hyphenated_text = '-'.join(coref_list)
#         coref_hyphenated_text = re.sub('[^A-Za-z0-9-]+', ' ', coref_hyphenated_text).strip()
#         coref_spaced_text = ' '.join(coref_list)
#         coref_spaced_text = re.sub('[^A-Za-z0-9-]+', ' ', coref_spaced_text).strip()
#         coref_resolved_text = coref_resolved_text.replace(coref_spaced_text, coref_hyphenated_text)
#     return coref_resolved_text

In [21]:
def extract_candidate_terms(text):
    """Extracts all nouns from text as aspect term candidates"""
    
    req_tag = ['NN', 'NNPS']
    candidate_terms = []
    
    for token in nlp(text):
        # print(token, token.tag_)
        if (
            token.tag_ in req_tag and # token must be noun
            token.shape_ != 'x' and # token mustn't be only one character
            token.shape_ != 'xx' and # token mustn't be only two character
            token.shape_ != 'xxx' # token mustn't be only three character
        ):
            candidate_terms.append(token.lemma_)
            
    return candidate_terms

In [ ]:
# # Resolve Coreferences
# df['text'] = df['text'].apply(lambda x: resolve_coreference(x) if len(x.split())>4 else x)

In [ ]:
# # Extract aspect term candidates
# df['aspect_term_candidates'] = df.text.apply(extract_candidate_terms)

In [22]:
df.head(10)

,text,terms,aspects
0,But the staff was so horrible to us.,"[(staff, negative)]","[(service, negative)]"
1,"To be completely fair, the only redeeming fact...","[(food, positive)]","[(food, positive), (anecdotes/miscellaneous, n..."
2,"The food is uniformly exceptional, with a very...","[(food, positive), (kitchen, positive), (menu,...","[(food, positive)]"
3,Where Gabriela personaly greets you and recomm...,[],"[(service, positive)]"
4,"For those that go once and don't enjoy it, all...",[],"[(anecdotes/miscellaneous, positive)]"
5,"Not only was the food outstanding, but the lit...","[(food, positive), (perks, positive)]","[(food, positive), (service, positive)]"
6,It is very overpriced and not very tasty.,[],"[(food, negative), (price, negative)]"
7,Our agreed favorite is the orrechiete with sau...,"[(orrechiete with sausage and chicken, positiv...","[(food, positive), (service, positive)]"
8,The Bagels have an outstanding taste with a te...,"[(Bagels, positive)]","[(food, positive)]"
9,Nevertheless the food itself is pretty good.,"[(food, positive)]","[(food, positive)]"


## Domain Specific Embedding Approach - Opinion Word Mining

In [32]:
class OpinionWordMiner:
    def __init__(self, tokenizer, gamma = 0.0005, we_layer_list=[-1]):
        self.tokenizer = tokenizer
        self.gamma = gamma
        
        
    def align_subwords_embeddings(self, marked_text, orig_word_embeddings):
        """
        This function merges the word embeddings of subwords from BERT tokenizer and averages them.
        """
        subword_token_ids = self.tokenizer(marked_text).word_ids()[1:-1]
        # print(subword_token_ids)
        we = np.zeros((len(set(subword_token_ids)), orig_word_embeddings.shape[1]))
        for i, idx in enumerate(subword_token_ids):
            # print(i, idx)
            if np.any(we[idx, :]):
                we[idx, :] = np.mean(np.vstack((we[idx, :], orig_word_embeddings[i, :])), axis=0)
            else:
                we[idx, :] = orig_word_embeddings[i, :]
        return we

    def get_word_embeddings(self, marked_text):
        """
        This function extracts the word/token embeddings from the specified hidden layers."""
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
            
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)

        token_vecs_cat = []
        for token in token_embeddings:
            cat_vec = torch.cat((
                token[0],
                token[-1]
            ), dim=0)
            token_vecs_cat.append(cat_vec.detach().numpy())
        return np.array(token_vecs_cat), tokenized_text
    
    def mine_opinion_words(self, text, aspect_word):
        idx2word_map = defaultdict(list)
        we_tokens_pos_tags, we_tokens_deps = [], []
        
        marked_text = "[CLS] " + text + " [SEP]"
        wts, _ = self.get_word_embeddings(marked_text)
        # print(wts.shape)
        word_emb = self.align_subwords_embeddings(marked_text, wts)

        for token, subword_idx in zip(_, self.tokenizer(marked_text).word_ids()[1:-1]):
            idx2word_map[subword_idx].append(token.replace('#', ''))

        for key, val in idx2word_map.items():
            idx2word_map[key] = ''.join(val)

        word2idx_map = {val:key for key, val in idx2word_map.items()}
        # print(word2idx_map)
        spacy_tokens_pos_tags = {token.text.lower():token.pos_ for token in nlp(text)}
        spacy_tokens_deps = {token.text.lower():token.dep_ for token in nlp(text)}

        for we_tokens in list(idx2word_map.values()):
            if we_tokens in spacy_tokens_pos_tags:
                we_tokens_pos_tags.append(spacy_tokens_pos_tags[we_tokens])
                we_tokens_deps.append(spacy_tokens_deps[we_tokens])
            else:
                we_tokens_pos_tags.append(None)
                we_tokens_deps.append(None)

        word_i = word2idx_map[aspect_word]
        # print('WI: ', word_i)
        dep_df = pd.DataFrame(rbf_kernel(word_emb, word_emb, self.gamma)[word_i], columns = ['dep'], 
                              index = idx2word_map.values())
        dep_df['pos'] = we_tokens_pos_tags
        dep_df['deps'] = we_tokens_deps
        dep_df = dep_df.reset_index().rename(columns = {'index':'opwo'})
        display(dep_df)
        asp_word_dep = dep_df[(dep_df['opwo'] == aspect_word)]['deps'].values[0]
        # print('ASP DEP: ', asp_word_dep)
        if asp_word_dep in ['dobj', 'pobj', 'conj']:
            display(dep_df[
#                 (dep_df['pos'] == 'ADJ')|\
                                  (dep_df['deps'].isin(['amod', 'dobj']))].sort_values(by = 'dep', 
                                                                          ascending = False))
            opinion_word = dep_df[
#                 (dep_df['pos'] == 'ADJ')|\
                                  (dep_df['deps'].isin(['amod', 'dobj']))].sort_values(by = 'dep', 
                                                                          ascending = False).head(1).index.values[0]
        else:
#             nsubj:acomp
            display(dep_df[
#                 (dep_df['pos'] == 'ADJ')|\
                                  (dep_df['deps'].isin(['acomp']))].sort_values(by = 'dep', 
                                                                          ascending = False))
            opinion_word = dep_df[
#                 (dep_df['pos'] == 'ADJ')|\
                                  (dep_df['deps'].isin(['acomp']))].sort_values(by = 'dep', 
                                                                          ascending = False).head(1).index.values[0]
            
        #left check
        l1text, l2text = '', ''
        try:
            if dep_df.iloc[opinion_word-1]['pos'] == 'PART':
                l1text = dep_df.iloc[opinion_word-1]['opwo'] + ' '
        except:
            pass
        try:
            if dep_df.iloc[opinion_word1-2]['pos'] == 'PART':
                l2text = dep_df.iloc[opinion_word-2]['opwo'] + ' '
        except:
            pass
        
        # Right check
        r1text, r2text = '', ''
        try:
            if dep_df.iloc[opinion_word+1]['pos'] == 'PART':
                r1text = ' ' + dep_df.iloc[opinion_word+1]['opwo']
        except:
            pass
        try:
            if dep_df.iloc[opinion_word1+2]['pos'] == 'PART':
                r2text = ' ' + dep_df.iloc[opinion_word+1]['opwo']
        except:
            pass
        return l2text + l1text + dep_df.iloc[opinion_word]['opwo'] + r1text + r2text 

In [ ]:
exp(-gamma * ||x-y||^2)

In [33]:
owm = OpinionWordMiner(tokenizer)

In [38]:
# Testing on text inputs
# text, asp_word = df.loc[8, 'text'], 'bagels'
# text, asp_word = "The cab ride was amazing but the driver was pathetic.", 'ride'
# text, asp_word = "The cab ride was amazing but the driver was pathetic.", 'driver'
# text, asp_word = "DelhiPalace has probably the best service and their chicken is yummy.", 'chicken'
text, asp_word = "DP has probably the best service but their chicken is yummy.", 'service'
# text, asp_word = "I goto Olive Garden for their beautiful ambience and calming music.", 'music'
# text, asp_word = "I goto Olive Garden for their beautiful ambience and calming music.", 'ambience'
# text, asp_word = "Cafe Bistro had very bad waitress, but I got there just for their amazing coffee.", 'waitress'
# text, asp_word = "Cafe Bistro had very bad waitress, but I got there just for their amazing coffee.", 'coffee'
# text, asp_word = "The food joint had some issues with their payments, but ambience was topnotch.", 'payments'
# text, asp_word = "The food joint had some issues with their payments, but ambience was topnotch.", 'ambience'
# text, asp_word = "I went to watch a movie. The actor was not good.", 'actor'

# text = resolve_coreference(text)
print(text, asp_word)

owm.mine_opinion_words(text, asp_word)

DP has probably the best service but their chicken is yummy. service


,opwo,dep,pos,deps
0,[CLS],0.681804,None,None
1,dp,0.758891,PROPN,nsubj
2,has,0.727692,VERB,ROOT
3,probably,0.676997,ADV,advmod
4,the,0.707547,DET,det
5,best,0.695795,ADJ,amod
6,service,1.000000,NOUN,dobj
7,but,0.695976,CCONJ,cc
8,their,0.721810,PRON,poss
9,chicken,0.697497,NOUN,nsubj


,opwo,dep,pos,deps
6,service,1.000000,NOUN,dobj
5,best,0.695795,ADJ,amod


'service'

In [ ]:
        ride driver . . . 
ride
driver
.
.
.

In [ ]:
## RULES 
# dobj:amod
# nsubj:acomp

In [ ]:
# # Testing on text inputs
# text = df.loc[14, 'text']
# # text = "The cab ride was amazing but the driver was pathetic."
# # text = "Delhi palace has probably the best service and their chicken is mindblow."

# text = resolve_coreference(text)
# mark_text = "[CLS] " + text + " [SEP]"
# print('Text: ', mark_text)

# # Get word embeddings from BERT domain specific model
# wts, _ = get_word_embeddings(mark_text)

# # Algining word embeddings for subwords by averaging
# word_emb = align_subwords_embeddings(mark_text, tokenizer, wts)

# idx2word_map = defaultdict(list)
# for token, subword_idx in zip(_, tokenizer(mark_text).word_ids()[1:-1]):
#     idx2word_map[subword_idx].append(token.replace('#', ''))

# for key, val in idx2word_map.items():
#     idx2word_map[key] = ''.join(val)

# word2idx_map = {val:key for key, val in idx2word_map.items()}  
    
# we_tokens_pos_tags, we_tokens_deps = [], []
# spacy_tokens_pos_tags = {token.text.lower():token.pos_ for token in nlp(text)}
# spacy_tokens_deps = {token.text.lower():token.dep_ for token in nlp(text)}

# for we_tokens in list(idx2word_map.values()):
#     if we_tokens in spacy_tokens_pos_tags:
#         we_tokens_pos_tags.append(spacy_tokens_pos_tags[we_tokens])
#         we_tokens_deps.append(spacy_tokens_deps[we_tokens])
#     else:
#         we_tokens_pos_tags.append(None)
#         we_tokens_deps.append(None)

# print(word2idx_map)
# word_i = 8
# dep_df = pd.DataFrame(rbf_kernel(word_emb, word_emb, gamma)[word_i], columns = ['dep'], 
#                       index = idx2word_map.values())
# dep_df['pos'] = we_tokens_pos_tags
# dep_df['deps'] = we_tokens_deps
# dep_df[dep_df['pos'] == 'ADJ'].sort_values(by = 'dep', ascending = False)

In [ ]:
labels = {'kev', 'ben', 'sc'}

In [ ]:
sorted(att_scores.items(), key = lambda x: x, reverse = True)

In [ ]:
token_we, tok_text = get_word_embeddings(mark_text)

## Rule Based Approach

In [ ]:
ii = 9
tt = df.loc[ii, 'text']
tt = 'The bagels have an oustanding taste'
asp = df.loc[ii, 'terms']
print(tt, asp)
doc = nlp(tt)
print([to_nltk_tree(sent.root).pretty_print() for sent in doc.sents])

dep_parser = []
for token in doc:
    dependency, governer_word, governer_pos, dependent_word, dependent_pos = token.dep_, token.head, token.head.pos_, token, token.pos_
    dep_parser.append([dependency, governer_word, governer_pos, dependent_word, dependent_pos])
    
# displacy.serve(doc, style='dep')

In [ ]:
pd.DataFrame(dep_parser)

In [ ]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [ ]:
tr = to_nltk_tree(list(doc.sents)[0].root)

In [ ]:
def traverse_tree(tree):
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverse_tree(subtree)

In [ ]:
traverse_tree(tr)

In [ ]:
# ii = 8
# tt = df.loc[ii, 'text']
# asp = df.loc[ii, 'terms']
# asp_term_sel = 'Bagels'
# print(tt, asp)
# doc = nlp(tt)
# print([to_nltk_tree(sent.root).pretty_print() for sent in doc.sents])


# predicates = {}
# for token in doc:
#     dependency, governer_word, governer_pos, dependent_word, dependent_pos = token.dep_, token.head, token.head.pos_, token, token.pos_

# #     print(dependency, governer_word, governer_pos, dependent_word, dependent_pos)
    
#     if dependent_pos == 'ADJ':
#         predicates[dependent_word] = [dependent_word]
    
#     if dependency == 'nsubj':
#         sub = dependent_word
#         conn = 
#         print('SUB: ', sub)
#     if dependency == 'dobj':
#         print(dependency, governer_word, governer_pos, dependent_word, dependent_pos)
#         obj = dependent_word
#         print('OBJ: ', obj)
        
#     if obj in predicates:
#         predicate = predicates[dependent_word]
#         print('PRED: ', predicate)
    
# displacy.serve(doc, style='dep')

In [ ]:
# ii = 9
# tt = df.loc[ii, 'text']
# asp = df.loc[ii, 'terms']
# print(tt, asp)
# doc = nlp(tt)
# displacy.serve(doc, style='dep')

In [ ]:
# # Run dependency parser and extract opinion words that are more than given threshold
# doc = stanzanlp('the bagels were outstanding as they were gummy and chewy.')

# sentiment_score = defaultdict(int)
# for sent in doc.sentences:
#     for word in sent.words:
#         if word.deprel == "flat:foreign":
#             print(word.text, sent.words[word.head-1].text, TextBlob(word.text).sentiment.polarity)
#             if TextBlob(word.text).sentiment.polarity>=0.5 or TextBlob(word.text).sentiment.polarity<=-0.5:
#                 sentiment_score[sent.words[word.head-1].text] = TextBlob(word.text).sentiment.polarity